## Module 2: Conditional Generative Adversarial Networks

(Link to paper: https://arxiv.org/abs/1411.1784)

In Conditional GANs, we feed a 'condition' to both the generator and discriminator during training. This will allow us to generate images fulfilling a condition, by feeding the condition to the generator, along with the random noise.

![](CGAN.png)

We will use MNIST data for this experiment. We will try to condition the network using the class labels of MNIST data. Thus we should be able to generate images from a particular class.

In [ ]:
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable

%matplotlib inline


mb_size = 64
Z_dim = 100
X_dim = 784
y_dim = 10
h_dim = 128
cnt = 0
lr = 1e-3


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    vec = torch.randn(*size) * xavier_stddev
    if setcuda:
        vec = vec.cuda()
    return Variable(vec, requires_grad=True)

setcuda = True


We will feed the condition as a 'one hot vector encoding' of the class label. The functions below load the MNIST dataset and convert the label into a one hot encoded vector.

In [ ]:
import torchvision
import torchvision.transforms as transforms

# load the dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0,), (1,))])
trainset = torchvision.datasets.MNIST(root='../../data/lab3', train=True,
                                        download=False,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=mb_size,
                                          shuffle=True, num_workers=2)
classes = ('0','1','2','3','4','5','6','7','8','9')

def onehotencoder(x):
    y = np.zeros((x.numpy().shape[0],10))
    for i in range(x.numpy().shape[0]):
        y[i,x[i]] = 1
    return y

dataiter = iter(trainloader)
dataiter = iter(trainloader)
images, labels = dataiter.next()

def mnist_next(dataiter):

    try:
        images, labels = dataiter.next()
        labels = onehotencoder(labels)
        images = images.view(images.numpy().shape[0],28*28)
    except:
        dataiter = iter(trainloader)
        images, labels = dataiter.next()
        labels = onehotencoder(labels)
        images = images.view(images.numpy().shape[0],28*28)
    return images.numpy(), labels

def initialize_loader(trainset):
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=mb_size,
                                          shuffle=True, num_workers=2)
    dataiter = iter(trainloader)
    return dataiter

images, labels = mnist_next(dataiter)
# print some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()
img = torchvision.utils.make_grid(images)
npimg = img.numpy()
plt.imshow(np.transpose(npimg, (1, 2, 0)))

In order to pass the condition to the generator and discriminator, we have to represent them in a 'joint vector representation'. For our joint vector representation, we will concatenate the one-hot encoded vector with the noise vector.

In [ ]:
""" ==================== GENERATOR ======================== """

Wzh = xavier_init(size=[Z_dim + y_dim, h_dim])
Whx = xavier_init(size=[h_dim, X_dim])

bzvar = torch.zeros(h_dim)
bhvar = torch.zeros(X_dim)

if setcuda:
    bzvar = bzvar.cuda()
    bhvar = bhvar.cuda()

bzh = Variable(bzvar, requires_grad=True)
bhx = Variable(bhvar, requires_grad=True)

def G(z, c):
    inputs = torch.cat([z, c], 1)
    h = nn.relu(torch.mm(inputs, Wzh) + bzh.repeat(inputs.size(0), 1))
    X = nn.sigmoid(torch.mm(h, Whx) + bhx.repeat(h.size(0), 1))
    return X


Similarly, for the discriminator, we will concatenate the one hot encoded vectors with the flattened image to get the joint vector representation.

In [ ]:
""" ==================== DISCRIMINATOR ======================== """

Wxh = xavier_init(size=[X_dim + y_dim, h_dim])
Why = xavier_init(size=[h_dim, 1])
bxvar = torch.zeros(h_dim)
bhvar = torch.zeros(1)
if setcuda:
    bxvar = bxvar.cuda()
    bhvar = bhvar.cuda()
    
bxh = Variable(bxvar, requires_grad=True)
bhy = Variable(bhvar, requires_grad=True)

def D(X, c):
    inputs = torch.cat([X, c], 1)
    h = nn.relu(torch.mm(inputs, Wxh) + bxh.repeat(inputs.size(0), 1))
    y = nn.sigmoid(torch.mm(h, Why) + bhy.repeat(h.size(0), 1))
    return y

Let us extract the parameters for training and create the ADAM solvers.

In [ ]:
G_params = [Wzh, bzh, Whx, bhx]
D_params = [Wxh, bxh, Why, bhy]
params = G_params + D_params

""" ===================== TRAINING ======================== """


def reset_grad():
    for p in params:
        p.grad.data.zero_()


G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

ones_label = Variable(torch.ones(mb_size))
zeros_label = Variable(torch.zeros(mb_size))

if setcuda:
    ones_label = ones_label.cuda()
    zeros_label = zeros_label.cuda()

Here, we do the actual training. It is similar to the vanilla GAN network, except that we concatenate the one-hot encoded labels to the inputs of both networks.

In [ ]:
dataiter = initialize_loader(trainset)

for it in range(1000):
    # Sample data
    z = Variable(torch.randn(mb_size, Z_dim))
    X, c = mnist_next(dataiter)
    if X.shape[0]!=mb_size:
        dataiter = initialize_loader(trainset)
        X,c = mnist_next(dataiter)
    X = Variable(torch.from_numpy(X))
    c = Variable(torch.from_numpy(c.astype('float32')))
    
    if setcuda:
        z = z.cuda()
        X = X.cuda()
        c = c.cuda()

    # Dicriminator forward-loss-backward-update
    G_sample = G(z, c)
    D_real = D(X, c)
    D_fake = D(G_sample, c)

    D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
    D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
    D_loss = D_loss_real + D_loss_fake

    D_loss.backward()
    D_solver.step()

    # Housekeeping - reset gradient
    reset_grad()

    # Generator forward-loss-backward-update
    if setcuda:
        z = Variable(torch.randn(mb_size, Z_dim).cuda())
    else:
        z = Variable(torch.randn(mb_size, Z_dim))
    G_sample = G(z, c)
    D_fake = D(G_sample, c)

    G_loss = nn.binary_cross_entropy(D_fake, ones_label)

    G_loss.backward()
    G_solver.step()

    # Housekeeping - reset gradient
    reset_grad()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'.format(it, D_loss.data.cpu().numpy(), G_loss.data.cpu().numpy()))

Let us see the images produced by the C-GAN. By changing the condition vector, we can generate images from different classes. 

In [ ]:
if setcuda:
    z = Variable(torch.randn(mb_size, Z_dim).cuda())
    discr_codes = Variable(torch.zeros(64,10).cuda())
else:
    z = Variable(torch.randn(mb_size, Z_dim))
    discr_codes = Variable(torch.zeros(64,10))

discr_codes[:,7] = 1 
samples = G(z,discr_codes)
samples = samples.cpu()
img = samples.data
img = img.view([64,1,28,28])
img = torchvision.utils.make_grid(img)
img = img.permute(1,2,0)
plt.imshow(img.numpy())

### Questions/Exercises:

<ol>
<li> What do the noise vectors mean? What happens when we gradually change the noise vectors? </li>
<li> How does the plot of error compare to vanilla GAN?</lo>
</ol>